In [1]:
import requests
import os
import pandas as pd
from io import StringIO
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import datetime
from datetime import timedelta
import os
from google.oauth2 import service_account
import pandas_gbq
import pytz

In [2]:
offers_by_accounts_offers = pd.read_excel('/ИнАпп/sql/offers_by_accounts.xlsx', sheet_name = 'offers')
Networks = pd.read_excel('/ИнАпп/sql/networks.xlsx', sheet_name = 'Networks')

Downloading: 100%|██████████|
    Event_Time Media_Source_x Event_Name geo_x     App_ID  Event_Pure  \
0   2025-02-02         AU iOS      event    UK  bet365 UK       15.52   
1   2025-02-02        Default      event    CA  bet365 CA      -25.61   
2   2025-02-02             UK      event    UK  bet365 UK     2025.06   
3   2025-02-02            UK1      event    UK  bet365 UK       22.77   
4   2025-02-02           UK16      event    UK  bet365 UK        0.00   
..         ...            ...        ...   ...        ...         ...   
85  2025-02-05           UK32      event    UK  bet365 UK       13.81   
86  2025-02-05           UK33      event    UK  bet365 UK       24.12   
87  2025-02-05            UK5      event    UK  bet365 UK       17.52   
88  2025-02-05            UK6      event    UK  bet365 UK       -2.94   
89  2025-02-05            UK9      event    UK  bet365 UK        0.00   

   Offer_Name   Brand Media_Source_y    geo_y  cpa curr   OS        manager  \
0   bet365 UK 

C:\Users\serge\AppData\Local\Temp\ipykernel_8440\4112463594.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for_dozap_bet365.rename(columns={'geo_x':'geo'},inplace=True)


In [4]:
# Appmetrica =====================================================================================================================================

offers_by_accounts_offers = pd.read_excel('/ИнАпп/sql/offers_by_accounts.xlsx', sheet_name = 'offers')

# AppMetrica
api_token = 'y0_AgAAAABnYUewAAozrAAAAAEOni-mAADsUBqHSzZPcZ9Wp4N12xznnLJIEQ'
headers = {"authorization": f"OAuth {api_token}"}

# Загружаем данные приложений
app_data = pd.read_excel(
    '/ИнАпп/sql/Not_AF_export/Adjust/appmetrica_apps.xlsx'
)

# Список для сохранения данных
data_frames = []

def appmetrica_api(url_, Offer_Name):
    try:
        # Выполняем запрос
        response = requests.get(url_, headers=headers)
        if response.status_code == 200:
            # Читаем CSV из ответа
            csv_data = pd.read_csv(StringIO(response.text))
            csv_data['Offer_Name'] = Offer_Name  # Добавляем колонку app_id для идентификации данных
            data_frames.append(csv_data)
        else:
            print(f"Ошибка при запросе для app_id:{Offer_Name}, статус: {response.status_code}")
    except Exception as e:
        print(f"Ошибка при обработке app_id:{Offer_Name}: {e}")

# запрос Cupis
appmetrica_api(
    Offer_Name = "cupis",
    url_=f"https://api.appmetrica.yandex.ru/v2/user/acquisition.csv?"
)
7
# запрос Wink IOS crm_subscr_purch_trial
appmetrica_api(
    Offer_Name = "Wink IOS",
    url_=f"https://api.appmetrica.yandex.ru/v2/user/acquisition.csv?"
)

# запрос Wink IOS crm_subscr_purch_notrial
appmetrica_api(
    Offer_Name = "Wink IOS",
    url_=f"https://api.appmetrica.yandex.ru/v2/user/acquisition.csv?"
)  

# запрос Wink AOS crm_subscr_purch_trial
appmetrica_api(
    Offer_Name = "Wink AOS",
    url_=f"https://api.appmetrica.yandex.ru/v2/user/acquisition.csv?"
)

# запрос Wink AOS crm_subscr_purch_notrial
appmetrica_api(
    Offer_Name = "Wink AOS",
    url_=f"https://api.appmetrica.yandex.ru/v2/user/acquisition.csv?"
)

# запрос Srochnodengy
appmetrica_api(
    Offer_Name = "Srochnodengy",
    url_=f"https://api.appmetrica.yandex.ru/v2/user/acquisition.csv?"
)

# запрос Adengi
appmetrica_api(
    Offer_Name = "Adengi",
    url_=f"https://api.appmetrica.yandex.ru/v2/user/acquisition.csv?"
)

# запрос TVOE
appmetrica_api(
    Offer_Name = "TVOE",
    url_=f"https://api.appmetrica.yandex.ru/v2/user/acquisition.csv?"
)

# запрос Carmoney
appmetrica_api(
    Offer_Name = "Carmoney",
    url_=f"https://api.appmetrica.yandex.ru/v2/user/acquisition.csv?"
)

# Конкатенируем все Датафреймы
if data_frames:
    df = pd.concat(data_frames, axis=0, ignore_index=True)
    print(df)
else:
    print("Нет данных для объединения.")

# Для проверки выгрузка
df.to_excel('/ИнАпп/sql/state_data/appmetrica_chek.xlsx')

# Там есть строки тотал, их убераем тут
df = df[df["Date"]!="Totals and averages"]


# Тут мы передаем бренды указанные вначале, и бренды, которые не равны им, но имеющие в поле Advertising Network Mobiads
df = df[~((df['Offer_Name'].isin(['cupis', 'TVOE'])) & (~df['Advertising Network'].str.contains('Mobiads', case=False, na=False)))]

# Тут мы передаем бренды указанные вначале, и бренды, которые не равны им, но имеющие в поле Advertising Network Mobiads
df = df[~((df['Offer_Name'].isin(['Wink IOS', 'Wink AOS'])) & (~df['Advertising Network'].str.contains('_rta', case=False, na=False)))]

# Тут мы передаем бренды указанные вначале, и бренды, которые не равны им, но имеющие в поле Campaign Mobiads
df = df[(df['Offer_Name'].isin(['Carmoney', 'Srochnodengy'])) | ((df['Campaign'].str.contains('MobiAds|Mobiads', case=False, na=False)) & \
    (~df['Offer_Name'].isin(['Carmoney', 'Srochnodengy'])))]


# Полуется так, что наши данные имеют столбцы с ивент неймами, и они не совсем как нам нужно располагаются (долго объяснять можно выгрузить
# и глянуть). Здесь мы переделываем нашу таблицу в нужный нам формат.
# тут обозначаем столбцы которые не будем менять
columns_to_keep = ['Date', 'Advertising Network', 'Campaign', 'Offer_Name']
# передаем в переменную цикл, которые будет проходится по названиям столбца, в названиях которые есть указанный текст - он обозначает ивен нейм
event_columns = [col for col in df.columns if "Number of devices with events" in col]

# Здесь сам метод который производит преобразования. Ивент нейм из названий в столбцах, распространиться на строки, 
# по значениям на которые он распространяется. Дальше мы в один столбец - ивент пюре закидываем все числовые значения по ивентнеймам, 
# которые ранее были в разных столбцах.
df_melt = df.melt(
    id_vars=columns_to_keep,
    value_vars=event_columns,
    var_name='Event_Name',
    value_name='Event_Pure'
)

# Удаляются строки с пустыми значениями в строках столбца ивент пюре
df_melt = df_melt.dropna(subset=['Event_Pure']).reset_index(drop=True)
# Устраняются строки с нулевыми значениями в строках столбца ивент пюре
df_melt = df_melt[df_melt['Event_Pure']!=0]
# По полю ивент нейм устраняются в названии ивентов - в строках текст указанные ниже, и удаляются пробелы
df_melt['Event_Name'] = df_melt['Event_Name'].str.replace("Number of devices with events", "", regex=False).str.strip()
# Устраняются так же одинарные кавычки, и устраняются снова пробелы
df_melt['Event_Name'] = df_melt['Event_Name'].str.replace("'", "", regex=False).str.strip()

# Создаем поле гео где везде RU - потому что в аппметрике льются только ру бренды
df_melt['geo'] = "RU"

# Создаем поле которому передает значения - ибо для брендов из аппметрики они идеентичны 
df_melt['App_ID'] = df_melt['Offer_Name']
# Переименовываем названия столбцов
df_melt.rename(columns={'Date':'Event_Time', 'Campaign':'Media_Source'}, inplace = True)

# Переименовываем название столбца в оферс бай аккаунт, чтобы смерджилось все
offers_by_accounts_offers.rename(columns={'App_Id':'App_ID'}, inplace=True)

# Инвент тайм превращаем в числовой формат
df_melt['Event_Time'] = pd.to_datetime(df_melt['Event_Time'])


df_melt.to_csv('/ИнАпп/sql/state_data/appmetrica/appmetrica1.csv', index=False)

# Удаляем дубликаты в по этим столбцам (дубликаты уникальных сочетаний этих столбцов)
columns_to_check_offers = ['App_ID', 'Offer_Name', 'Brand', 'Event_Name', 'Media_Source', 'geo', 'cpa', 'curr']
# Обрабатываем поле для дальнейшего удаления дубликатов
offers_by_accounts_offers = offers_by_accounts_offers.drop_duplicates(subset=columns_to_check_offers)

# Обрабатываем поле для дальнейшего удаления дубликатов
df_melt = df_melt.groupby(['Event_Time', 'Advertising Network', 'Media_Source', 'Offer_Name', 'Event_Name', 'geo', 'App_ID'], as_index=False)['Event_Pure'].sum()
#df_melt.to_csv('/ИнАпп/sql/state_data/appmetrica/appmetrica1.csv', index=False)
# Объединение данных с офферс бай аккаунт чтобы получить ставки которые платят нам и другие данные
df_melt = pd.merge(df_melt, offers_by_accounts_offers, on=['App_ID','Event_Name','geo'], how = 'left')

# Читаем данные из листа файла по заполненным сеткам российских брендов
Networks = pd.read_excel('/ИнАпп/sql/networks.xlsx', sheet_name = 'Networks')

# Переименовываем столбцы
df_melt.rename(columns={'Offer_Name_x':'Offer_Name', 'Media_Source_x':'Media_Source'},inplace=True)

# Тут оставляем столбцы, чтобы удалить по их уникальным сочетаниям дубли
columns_to_check_networks = ['Offer_Name', 'Media_Source', 'network_name', 'affiliate', 'Event_Name', 'payout', 'geo']

# Удаляем дуликаты
Networks = Networks.drop_duplicates(subset=columns_to_check_networks)

# Группируем данные для избежания дублирования данных при мердже
df_melt = df_melt.groupby(['Event_Time','Advertising Network','Media_Source','Offer_Name','Event_Name','geo','App_ID',\
    'Brand','cpa','curr','OS','manager'], as_index=False)['Event_Pure'].sum()

# Объединяем нетворкс и наши сырые данные чтобы получить ставки которые платим мы
df_merge = pd.merge(df_melt, Networks, on=['Offer_Name','Media_Source','Event_Name'], how='left', indicator=True)

# Оставляем только те значения которые не сопоставились чтобы дозаполнить их
for_dozap = df_merge[df_merge['_merge'] == 'left_only']

# Переименовываем столбец
for_dozap.rename(columns={'geo_x':'geo'},inplace=True)

# Оставляем нужныем нам столбцы и заполняем пустые значения значением RU
for_dozap = for_dozap[['Offer_Name', 'Media_Source', 'network_name', 'affiliate', 'Event_Name', 'payout', 'geo']]
for_dozap['geo'] = for_dozap['geo'].replace(['', 'NAN'], 'RU').fillna('RU')

# Тут объединяем appmetrica дозап и bet365
for_dozap = pd.concat([for_dozap, for_dozap_bet365], axis=0)

# Удаляем дубли
for_dozap = for_dozap.drop_duplicates(subset=columns_to_check_networks)

# Выгружаем для заполнения сеток
for_dozap.to_excel('/ИнАпп/sql/state_data/appmetrica/for_dozap.xlsx', index=False)

# Переименовываем столбец
df_merge.rename(columns={'geo_x':'geo'},inplace=True)
# Оставляем только те значения которые сопоставились чтобы отправить их в стату
df_merge = df_merge[df_merge['_merge'] == 'both']

# Группируем данные
df_merge = df_merge.groupby(
    ['Event_Time', 'Media_Source', 'Offer_Name', 'Event_Name', 'geo', 'App_ID', 'Brand', 'cpa', 'curr', 'OS', 
     'manager', 'network_name', 'affiliate', 'payout'],
    as_index=False
).agg(Conversion=('Event_Pure', 'sum'))

# Тут объединяем конечные данные по Appmetrica и bet365
df_merge = pd.concat([df_merge, bet365], axis=0)

# Выгружаем данные
df_merge.to_csv('/ИнАпп/sql/state_data/appmetrica/appmetrica.csv', index=False)

                    Date Advertising Network  \
0    Totals and averages                 NaN   
1             2025-02-02       Xiaomi Global   
2             2025-02-04       Xiaomi Global   
3             2025-02-03       Xiaomi Global   
4             2025-02-05            BIGO Ads   
..                   ...                 ...   
667           2025-02-05    Рассылки Кармани   
668           2025-02-05           Bidrunner   
669           2025-02-05           Bidrunner   
670           2025-02-05            MobZilla   
671           2025-02-05     AppSpire Agency   

                                       Campaign  \
0                                           NaN   
1                 Perfom_Xiaomi_Mobiads_Android   
2                 Perfom_Xiaomi_Mobiads_Android   
3                 Perfom_Xiaomi_Mobiads_Android   
4                         Cupis_Mobiads_BigoAds   
..                                          ...   
667                 LKKPTS_CarMoney_iOS/android   
668     CPA_Bid

C:\Users\serge\AppData\Local\Temp\ipykernel_8440\1511644540.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for_dozap.rename(columns={'geo_x':'geo'},inplace=True)
